In [ ]:
import eos
import pyhf
import json
import yaml
import numpy as np

# Analyzing [`pyhf`](https://pyhf.readthedocs.io) likelihoods with `EOS`

[`pyhf`](https://pyhf.readthedocs.io) has become a statistical tool, widely used in the experimental HEP community. It implements the HistFactory statistical model and has the nice feature of a distributable likelihood format with `JSON` files.

In this notebook we demonstrate how to add [`pyhf`](https://pyhf.readthedocs.io) likelihoods to `EOS` analyses.

For the purpose of this example, we construct a simple [`pyhf`](https://pyhf.readthedocs.io) analysis of a $B \to \pi l \nu$ signal decay. The analysis consists of one signal and one background channel with a background uncertainty, correlated across bins.

In [ ]:
model = pyhf.simplemodels.correlated_background(
            signal=[500, 800], bkg=[5000, 6000], bkg_down=[4900, 5800], bkg_up=[5100, 6200]
        )

data = [6000, 7200]

workspace = pyhf.Workspace.build(model, data)

# dump to json
with open('workspace_corr.json', 'w') as f:
    json.dump(workspace, f, indent=2)

workspace.model().spec

We can use [`pyhf`](https://pyhf.readthedocs.io) to fit the model parameters, which are two in this case: `mu` the signal strength, or scaling parameter for the signal, and `correlated_bkg_uncertainty` the correlated background nuisance parameter.

In [ ]:
best_fit = pyhf.infer.mle.fit(data + model.config.auxdata, model)
for name, value in zip(model.config.par_names, best_fit):
    print(f'{name}: {value}')

Since we measure more `data` as our `signal` +`background` event counts, we obtain a signal strength `mu`>1.

We add a [`pyhf`](https://pyhf.readthedocs.io) likelihood to an `eos.AnalysisFile` by adding a [`pyhf`](https://pyhf.readthedocs.io) type likelihood:

```yaml
likelihoods:
  - name: EXP-pyhf
    pyhf:
      file: workspace_corr.json
      parameter_map:
        mu: {'name': 'B->pilnu::mu', 'kinematics': {'q2_min': 1.0e-7, 'q2_max': 25.0}}
```
This likelihood is specified by the `file`, corresponding to the `pyhf workspace`, and an optional `parameter_map` to identify [`pyhf`](https://pyhf.readthedocs.io) parameters with `EOS` parameters or observables.

The optional `parameter_map` is a dictionary where the key is the parameter name on the [`pyhf`](https://pyhf.readthedocs.io) side and the value is either a
- `string`: This will be associated with an `EOS` parameter, if it exists. Alternatively, a new parameter will be created.
- `dict`: This will be associated with an `EOS` observable. The `dict` consists of a `name` (mandatory) and `kinematics`, `options` (optional).

In the above case we want to associate the signal strength `mu` with an `EOS` observable,
```yaml
observables:
  "B->pilnu::mu":
    latex: "$\\mu$"
    unit: '1'
    options: {}
    expression: "<<B->pilnu::BR;model=WET>> / <<B->pilnu::BR;model=SM>>"

```
This observable corresponds to the WET $B \to \pi l \nu$ branching ratio, relative to the SM.

We want to infer the posterior of the (lepton flavor independent, see `pyhf_corr.yaml`) `ublnul::Re{cVL}` Wilson coefficient, for which we add a prior:
```yaml
priors:
  - name: WET
    descriptions:
      - { 'parameter': 'ublnul::Re{cVL}', 'min':  0.5, 'max': 2.0, 'type': 'uniform' }
```
Constrained [`pyhf`](https://pyhf.readthedocs.io) parameters are automatically assigned the corresponding priors.

In [ ]:
af = eos.AnalysisFile("pyhf_corr.yaml")
af

We have added two posteriors. `EXP-pyhf` for the pure [`pyhf`](https://pyhf.readthedocs.io) model with two parameters and `WET-pyhf` including additional $B \to \pi l nu$ experimental constraints and $B \to \pi$ form factor constraints.

We can now use the `nested_sampling` capabilities of `EOS` to sample from these posteriors:

In [ ]:
eos.tasks.sample_nested(af, 'EXP-pyhf', base_directory='./', bound='multi', nlive=100, dlogz=9.0, maxiter=4000)

Note that the above command uses an unreasonably large value for `dlogz` (9.0) and small values for `maxiter` (4000), which is only done for the sake of this example.
In practice, you should use a smaller value for `dlogz` at about 1% of the log-evidence. For `maxiter`, you should use a value that is large enough to ensure that the sampler has converged.
Ideally, no value for `maxiter` should be required.

In [ ]:
eos.tasks.corner_plot(af, 'EXP-pyhf')

We can also extract the estimated value of $C_{V_L}$:

In [ ]:
parameter_samples = eos.data.ImportanceSamples('./data/EXP-pyhf/samples')

mean = np.average(parameter_samples.samples[:,0], weights = parameter_samples.weights)
std = np.sqrt(np.average((parameter_samples.samples[:,0]-mean)**2, weights = parameter_samples.weights))

print(f'$C_{{V_L}}$ = {mean:.4f} +/- {std:.4f}')

Due to the excess we found in the measured data, we see a slight increase in $C_{V_L}$. The value of the nuisance parameter is slightly pulled to a negative value, due to the larger signal contribution in the second bin of the [`pyhf`](https://pyhf.readthedocs.io) analysis.

We can also compare the posterior mode to the best fit point from above.

In [ ]:
eos.tasks.find_mode(af, 'EXP-pyhf', base_directory='./')

In [ ]:
mode = eos.data.Mode('./data/EXP-pyhf/mode-default')
print(f'EOS posterior mode: $C_{{V_L}} = {mode.mode[0]:.3f}')
print(f'pyhf best fit: $C_{{V_L}} = {np.sqrt(best_fit[1]):.3f}')

Next, we look at the `WET-pyhf` posterior, including additional measurements and properly accounting for form factor uncertainties.

In [ ]:
eos.tasks.sample_nested(af, 'WET-pyhf', base_directory='./', bound='multi', nlive=100, dlogz=9.0, maxiter=4000)

In [ ]:
eos.tasks.corner_plot(af, 'WET-pyhf')

In [ ]:
parameter_samples = eos.data.ImportanceSamples('./data/WET-pyhf/samples')

mean = np.average(parameter_samples.samples[:,0], weights = parameter_samples.weights)
std = np.sqrt(np.average((parameter_samples.samples[:,0]-mean)**2, weights = parameter_samples.weights))

print(f'$C_{{V_L}}$ = {mean:.4f} +/- {std:.4f}')

We see that the value of $C_{V_L}$ is significantly smaller than in the previous case, due to the inclusion of the additional measurements. In return, the nuisance parameter is pulled to much larger values, to compensate for the smaller Wilson coefficient.